In [1]:
import os
import json
from google.cloud import bigquery
import pandas as pd
import dateparser
from history_page import HistoryPage
from profile_page import ProfilePage

In [2]:
def create_dataset(dataset_ref, location='US'):
    dataset = bigquery.Dataset(dataset_ref)
    try:
        dataset = client.create_dataset(dataset)
        print('\n-- Dataset {} created --'.format(dataset_id))
    except:
        print('\n-- Dataset {} already exists --'.format(dataset_id))
        
def check_table_exists(table_ref):
    try:
        client.get_table(table_ref)
        return True
    except:
        return False


In [3]:
def get_schema_from_json(json, schema=[]):
    input_json = json.copy()
    if not input_json:
        return schema

    cur = input_json.pop()
    name = cur['name']
    field_type = cur['type']
    mode = cur['mode']
    fields = [] if 'fields' not in cur else get_schema_from_json(cur['fields'], [])
    schema.append(bigquery.SchemaField(name=name, field_type=field_type, mode=mode, fields=fields))

    return get_schema_from_json(input_json, schema)

In [4]:
def divide_into_batches(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [5]:
def database_setup(configuration, client, dataset_id):
    dataset_ref = client.dataset(dataset_id)
    create_dataset(dataset_ref, location='US')
    
    for table_name in ['northrend', 'azeroth', 'lorderon']:
        fields = configuration.get('fields')
        schema = get_schema_from_json(fields, [])
        file_path = './data_backfill/{}.json'.format(table_name)
        try:
            with open(file_path, 'r') as f:
                data = json.loads(f.read())
            create_table(table_name, schema, data)
        except Exception:
            print('No data found')

In [6]:
def create_table(table_name, schema, data):
    table_ref = dataset_ref.table(table_name)
    table = bigquery.Table(table_ref, schema=schema)
    if check_table_exists(table_ref):
        print('\n-- Table {} already exists --\n'.format(table_ref))
        return
    
    table = client.create_table(table)
    batches = divide_into_batches(data, 10000)
    for batch in batches:
        errors = client.insert_rows(table, batch)
        try:
            assert errors == []
        except Exception:
            print(Exception, errors[0])
    print('\n-- Table {} created --\n'.format(table_name))

In [7]:
def update_table(table_name, new_games):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    errors = client.insert_rows(table, new_games)
    print('Uploaded {} new games'.format(len('new_games')))

## Dev records

In [8]:
# command = !record followgrubby romantichuman @northrend
# inputs = {player_one: followgrubby, player_two:romantichuman}

In [9]:
bigquery_credpath = os.path.abspath('/Users/cdaly/Box Sync/Daly, Christopher/Keys/BigQuery Reader Project-88493810ca62.json')
client = bigquery.Client.from_service_account_json(bigquery_credpath)
job_config = bigquery.LoadJobConfig()
job_config.skip_leading_rows = 1
job_config.autodetect = True
dataset_id = 'wc3'

## Get the most recent data for player one

In [10]:
def get_last_bq_date(client, player, server):
    query = ('''SELECT date FROM  `bigquery-reader-project.wc3.{}`,
              UNNEST(team_one) AS first
            WHERE 
                first IN ('{}') 
            ORDER BY date DESC LIMIT 1
        '''.format(server, player))

    job = client.query(query)
    results = list(job.result())
    if len(results) == 0:
        print('Player ({}) not in table ({}).'.format(player, server))
        return None
    row = results[0]
    last_bq_date = row.get('date')
    print('Last BigQuery date: {}'.format(last_bq_date))
    return last_bq_date

In [11]:
def get_games_between_players(client, players, server):
    query = ('''SELECT * FROM  `bigquery-reader-project.wc3.{}`,
              UNNEST(team_one) AS first,
              UNNEST(team_two) AS second
            WHERE 
                first IN ('{player_one}', '{player_two}') 
            AND 
                second in ('{player_one}', '{player_two}')
            ORDER BY date DESC
        '''.format(server, **players))

    job = client.query(query)
    results = job.to_dataframe().to_dict(orient='records')
    if len(results) == 0:
        print('No games between ({player_one}) and ({player_two}) in database.'.format(**players))
    return results

In [12]:
def get_new_games(player, server, last_bq_date=None):
    print('Looking for new games...')
    def no_more_dates():
        if last_bq_date is None:
            return False
        else:
            return current_date <= last_bq_date
        
    def no_more_pages():
        return page >= next_page
    
    new_games = []
    page = 1
    
    while True:
        
        history_page = HistoryPage(player, server, page)
        games = history_page.games
        next_page = history_page.next_page

        for d in games:
            current_date = dateparser.parse(d.get('date'))
            if no_more_dates():
                return new_games
            else:
                new_games.append(d)
        
        if no_more_pages():
            return new_games
        else:
            page = next_page

In [13]:
def calculate_record(data_input, games):
    df = pd.DataFrame(games)
    
    record = {}
    for player in data_input.get('players').values():
        record[player] = df['winner'].apply(lambda x: player == x[0]).sum()
    
    return record

In [14]:
def lambda_handler(data_input):
    bigquery_credpath = os.path.abspath('/Users/cdaly/Box Sync/Daly, Christopher/Keys/BigQuery Reader Project-88493810ca62.json')
    client = bigquery.Client.from_service_account_json(bigquery_credpath)
    job_config = bigquery.LoadJobConfig()
    job_config.skip_leading_rows = 1
    job_config.autodetect = True
    dataset_id = 'wc3'
    
    player = data_input.get('players').get('player_one')
    server = data_input.get('server')

    # Update current data.
    last_bq_date = get_last_bq_date(client, player, server)
    new_games = get_new_games(player, server, last_bq_date=last_bq_date)
    if len(new_games) > 0:
        update_table(server, new_games)
    else:
        print('No new games.')
    
    # Get current record.
    games = get_games_between_players(client, data_input.get('players'), server)
    if len(games) > 0:
        record = calculate_record(data_input, games)
        data = {'games': games, 'record': record}
        return data

In [15]:
data_input = {
    'server': 'northrend',
    'players': {
        'player_one': 'Rellik',
        'player_two': 'Misterwinner',
    }
}
player = data_input.get('players').get('player_one')
server = data_input.get('server')

lambda_handler(data_input)

Last BigQuery date: 2018-12-03 19:42:00
Looking for new games...


ReadTimeout: HTTPConnectionPool(host='classic.battle.net', port=80): Read timed out. (read timeout=5)

In [ ]:
history_page = HistoryPage(player, server)
print(history_page)

In [ ]:
list(history_page.games)

In [ ]:
games = history_page.games()
list(games)

In [ ]:
next_page = history_page.next_page
next_page

In [ ]:
# 2 cases:
# No new games.
# New games. Loop through game history until we have everything.